## Preprocessing and concatenating numeric data

In [39]:
import pandas as pd
import pytz

from datetime import datetime
from functools import reduce

from functions import ffill_nans

<br>

### CoinGecko

In [40]:
gecko_btc_data = pd.read_parquet('../../1_data_acquisition/coin_gecko/gecko_btc_data.parquet.gzip')
gecko_eth_data = pd.read_parquet('../../1_data_acquisition/coin_gecko/gecko_eth_data.parquet.gzip')

Transform timestamp to UTC

In [41]:
for df in [gecko_btc_data, gecko_eth_data]:
    df['timestamp'] = [datetime.strptime(datetime.fromtimestamp(i).date().strftime('%Y-%m-%d')+'+00:00', '%Y-%m-%d%z').timestamp() for i in df.index]

In [42]:
gecko_btc_data = gecko_btc_data.set_index('timestamp')
gecko_eth_data = gecko_eth_data.set_index('timestamp')

Drop columns

> Among others the price is dropped. CryptoCompare price is used as target instead of CoinGecko price. Reason is slightly longer timeframe of historical data (couple months) and better methodology (CCCAGG). Downside is that CoinGecko uses data from 639 exchanges and CryptoCompare only from 301. Nevertheless the average difference between the two price calculations is 0.38 %, so the choice has no big impact on the analysis 

In [43]:
gecko_btc_data = gecko_btc_data.drop(columns=[
    'btc_current_price',
    'btc_market_cap',
    'btc_reddit_average_posts_48h',
    'btc_reddit_average_comments_48h',
    'btc_facebook_likes',
])

gecko_eth_data = gecko_eth_data.drop(columns=[
    'eth_current_price',
    'eth_market_cap',
    'eth_reddit_average_posts_48h',
    'eth_reddit_average_comments_48h',
    'eth_facebook_likes',
])


Invert deletions since they're all negative numbers

In [44]:
gecko_btc_data['btc_deletions'] = -gecko_btc_data['btc_deletions']
gecko_eth_data['eth_deletions'] = -gecko_eth_data['eth_deletions']

Deal with zeros

In [45]:
for column in ['total_issues', 'closed_issues', 'total_volume']:
    gecko_btc_data['btc_' + column] = gecko_btc_data['btc_' + column].replace(0, float('nan'))
    gecko_eth_data['eth_' + column] = gecko_eth_data['eth_' + column].replace(0, float('nan'))

Impute nans with the previous value

In [46]:
btc_exclude_cols = [
    'btc_total_issues',
    'btc_closed_issues',
]

eth_exclude_cols = [
    'eth_total_issues',
    'eth_closed_issues',
]

gecko_btc_data = ffill_nans(gecko_btc_data, exclude_cols=btc_exclude_cols)
gecko_eth_data = ffill_nans(gecko_eth_data, exclude_cols=eth_exclude_cols)

Remove two unrealistic values from ETH dataset

In [47]:
gecko_eth_data.loc[1521504000,'eth_twitter_followers'] = float('nan')
gecko_eth_data.loc[1521590400,'eth_twitter_followers'] = float('nan')
gecko_eth_data.loc[1490486400,'eth_total_issues'] = float('nan')
gecko_eth_data.loc[1490486400,'eth_closed_issues'] = float('nan')
gecko_eth_data.loc[1490572800,'eth_total_issues'] = float('nan')
gecko_eth_data.loc[1490572800,'eth_closed_issues'] = float('nan')
gecko_eth_data.loc[1548547200,'eth_total_issues'] = float('nan')
gecko_eth_data.loc[1548633600,'eth_total_issues'] = float('nan')

<br>

### CryptoCompare

In [48]:
cc_btc_data = pd.read_parquet('../../1_data_acquisition/crypto_compare/btc_data.parquet.gzip')
cc_eth_data = pd.read_parquet('../../1_data_acquisition/crypto_compare/eth_data.parquet.gzip')
cc_bvin_hourly = pd.read_parquet('../../1_data_acquisition/crypto_compare/btc_volatility_hourly.parquet.gzip')
cc_indices = pd.read_parquet('../../1_data_acquisition/crypto_compare/indices_data.parquet.gzip')

Drop columns (this includes PoW related variables for ETH since the goal is predicting future prices which are based on PoS)

In [49]:
cc_btc_data = cc_btc_data.drop(columns=[
    'btc_price_high', # redundant
    'btc_price_low', # redundant
    'btc_price_open', # redundant
    'btc_exchange_Bitci_volumeto', # too many NAs
    'btc_exchange_Bitci_volumefrom', # too many NAs
    'btc_exchange_Bitci_volumetotal', # too many NAs
    'btc_block_height', # redundant
    'btc_transaction_count_all_time', # redundant
])

cc_eth_data = cc_eth_data.drop(columns=[
    'eth_price_high', # redundant
    'eth_price_low', # redundant
    'eth_price_open', # redundant
    'eth_block_height', # redundant
    'eth_hashrate', # PoW variable
    'eth_difficulty', # PoW variable
    'eth_block_time', # PoW variable
    'eth_transaction_count_all_time', # redundant
])

Remove BTSE outlier on 25-02-2022

In [50]:
cc_btc_data.btc_exchange_BTSE_volumeto[1645747200] = float('nan')
cc_btc_data.btc_exchange_BTSE_volumefrom[1645747200] = float('nan')
cc_btc_data.btc_exchange_BTSE_volumetotal[1645747200] = float('nan')

Turn missing blockchain data in Sept 2022 showing zero into NA

In [51]:
columns = [
    'btc_new_addresses',
    'btc_active_addresses',
    'btc_transaction_count',
    'btc_large_transaction_count',
    'btc_average_transaction_value',
    'btc_hashrate',
    'btc_difficulty',
    'btc_block_time',
    'btc_block_size',
]

for column in columns:
    for time in [1663200000, 1663286400, 1663372800, 1663459200]:
        cc_btc_data[column][time] = float('nan')

Covert hourly BVIN data to daily

In [52]:
cc_bvin_hourly['date'] = [datetime.fromtimestamp(i, tz=pytz.utc).date() for i in cc_bvin_hourly.index]

In [53]:
drops = [
    'btc_volatility_index_high',
    'btc_volatility_index_low',
    'btc_volatility_index_open',
]

In [54]:
cc_bvin_daily = cc_bvin_hourly.drop(columns=drops).groupby('date').mean()
cc_bvin_daily.columns = ['index_BVIN_close']

In [55]:
cc_bvin_daily['timestamp'] = [datetime(i.year, i.month, i.day, tzinfo=pytz.utc).timestamp()  for i in cc_bvin_daily.index]
cc_bvin_daily = cc_bvin_daily.set_index('timestamp')

Clean and merge indices data

In [56]:
drops = [
    'index_MVDASC_high',
    'index_MVDASC_low',
    'index_MVDASC_open',
    'index_MVDASC_close',
    'index_MVDAMC_high',
    'index_MVDAMC_low',
    'index_MVDAMC_open',
    'index_MVDAMC_close',
    'index_MVDALC_high',
    'index_MVDALC_low',
    'index_MVDALC_open',
    'index_MVDALC_close',
    'index_MVDA_high',
    'index_MVDA_low',
    'index_MVDA_open',
]

cc_indices = cc_indices.drop(columns=drops)

In [57]:
cc_indices = pd.merge(cc_indices, cc_bvin_daily, left_index=True, right_index=True)

<br>

### Google Trends

In [58]:
gtrends_data = pd.read_parquet('../../1_data_acquisition/google_trends/google_trends.parquet.gzip')

In [59]:
for column in gtrends_data.columns:
    gtrends_data[column] = gtrends_data[column].astype(int)
    gtrends_data[column] = ((gtrends_data[column] / gtrends_data[column].shift(1)) - 1) * 100

gtrends_data = gtrends_data[1:]    
gtrends_data = gtrends_data[[not i for i in gtrends_data.index.duplicated()]]
gtrends_data = gtrends_data.replace(-100, float('nan')).replace(float('inf'), float('nan'))

In [60]:
gtrends_data = gtrends_data.add_prefix('gtrends_').add_suffix('_relative_change')

In [61]:
gtrends_data['timestamp'] = [datetime.strptime(i + '+00:00', '%Y-%m-%d%z').timestamp()  for i in gtrends_data.index]
gtrends_data = gtrends_data.set_index('timestamp')

<br>

### Yahoo Finance

In [62]:
yf_data = pd.read_parquet('../../1_data_acquisition/yahoo_finance/yf_data.parquet.gzip')

Deal with NAs

In [63]:
yf_data = ffill_nans(yf_data)

Transform date to timestamp

In [64]:
yf_data['timestamp'] = [datetime(i.year, i.month, i.day, tzinfo=pytz.utc).timestamp()  for i in yf_data.index]
yf_data = yf_data.set_index('timestamp')

<br>

### Technical indicators

In [65]:
btc_indicators = pd.read_parquet('btc_indicators.parquet.gzip')
eth_indicators = pd.read_parquet('eth_indicators.parquet.gzip')

<br>

### Concatenate all and save

Concatenate

In [66]:
btc_dfs = [
    gecko_btc_data,
    cc_btc_data,
    cc_indices,
    gtrends_data.drop(columns='gtrends_ethereum_relative_change'),
    yf_data,
    btc_indicators,
]

eth_dfs = [
    gecko_eth_data,
    cc_eth_data,
    cc_indices,
    gtrends_data.drop(columns='gtrends_bitcoin_relative_change'),
    yf_data,
    eth_indicators,
]

In [67]:
btc_numeric_data = pd.concat(btc_dfs, axis=1).sort_index()
eth_numeric_data = pd.concat(eth_dfs, axis=1).sort_index()

Impute the values of SP500, VIX and Gold on weekends

In [68]:
for column in ['sp500_price', 'sp500_volume', 'vix', 'gold_usd_price']:
    btc_numeric_data[column] = ffill_nans(btc_numeric_data[column])
    eth_numeric_data[column] = ffill_nans(eth_numeric_data[column])

Cut off data for which no price (target) is available yet

In [69]:
btc_numeric_data = btc_numeric_data[btc_numeric_data.index >= 1314403200]
eth_numeric_data = eth_numeric_data[eth_numeric_data.index >= 1445299200]

Ensure all columns are floats and downcast to smallest possible bit size

In [70]:
btc_numeric_data = btc_numeric_data.apply(lambda x: pd.to_numeric(x, downcast='float'))
eth_numeric_data = eth_numeric_data.apply(lambda x: pd.to_numeric(x, downcast='float'))

Save to parquet

In [71]:
btc_numeric_data.to_parquet('btc_numeric_data.parquet.gzip', compression='gzip')
eth_numeric_data.to_parquet('eth_numeric_data.parquet.gzip', compression='gzip')